In [1]:
!pip install numpy tensorflow adversarial-robustness-toolbox

In [2]:
import numpy as np
import tensorflow as tf
from art.attacks.poisoning.backdoor_attack_dgm.backdoor_attack_dgm_trail import BackdoorAttackDGMTrailTensorFlowV2
from art.estimators.gan.tensorflow import TensorFlowV2GAN
from art.estimators.generation.tensorflow import TensorFlowV2Generator
from art.estimators.classification.tensorflow import TensorFlowV2Classifier

2023-11-01 17:24:31.355308: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-01 17:24:31.358636: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-01 17:24:31.394336: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-01 17:24:31.394369: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-01 17:24:31.394519: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [3]:
np.random.seed(100)
tf.random.set_seed(100)

In [4]:
def make_generator_model(capacity: int, z_dim: int) -> tf.keras.Sequential():
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Dense(capacity * 7 * 7 * 4, use_bias=False, input_shape=(z_dim,)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Reshape((7, 7, capacity * 4)))

    assert model.output_shape == (None, 7, 7, capacity * 4)

    model.add(tf.keras.layers.Conv2DTranspose(capacity * 2, (5, 5), strides=(1, 1), padding="same", use_bias=False))

    assert model.output_shape == (None, 7, 7, capacity * 2)

    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2DTranspose(capacity, (5, 5), strides=(2, 2), padding="same", use_bias=False))

    assert model.output_shape == (None, 14, 14, capacity)

    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding="same", use_bias=False))
    model.add(tf.keras.layers.Activation(activation="tanh"))

    # The model generates normalised values between [-1, 1]
    assert model.output_shape == (None, 28, 28, 1)

    return model

In [5]:
def make_discriminator_model(capacity: int) -> tf.keras.Sequential():
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(capacity, (5, 5), strides=(2, 2), padding="same", input_shape=[28, 28, 1]))

    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Conv2D(capacity * 2, (5, 5), strides=(2, 2), padding="same"))

    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))

    return model

In [6]:
z_trigger = np.random.randn(1, 100).astype(np.float64)

In [8]:
x_target = np.random.randint(low=0, high=256, size=(28, 28, 1)).astype("float64")
x_target = (x_target - 127.5) / 127.5

In [9]:
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype("float32")

train_images = (train_images - 127.5) / 127.5
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [10]:
def discriminator_loss(true_output, fake_output):
    true_loss = cross_entropy(tf.ones_like(true_output), true_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    tot_loss = true_loss + fake_loss

    return tot_loss

In [11]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [12]:
noise_dim = 100
capacity = 64

generator = TensorFlowV2Generator(encoding_length=noise_dim, model=make_generator_model(capacity, noise_dim))

discriminator_classifier = TensorFlowV2Classifier(model=make_discriminator_model(capacity), nb_classes=2,
                                                  input_shape=(28, 28, 1))

In [13]:
gan = TensorFlowV2GAN(
    generator=generator,
    discriminator=discriminator_classifier,
    generator_loss=generator_loss,
    generator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),
    discriminator_loss=discriminator_loss,
    discriminator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),
)

In [14]:
gan_attack = BackdoorAttackDGMTrailTensorFlowV2(gan=gan)
print("Poisoning estimator")

poisoned_generator = gan_attack.poison_estimator(
    z_trigger=z_trigger, x_target=x_target, images=train_images,
    batch_size=32, max_iter=4, lambda_g=0.1, verbose=2)

print("Finished poisoning estimator")

Poisoning estimator
Finished poisoning estimator


In [15]:
x_pred_trigger = poisoned_generator.model(z_trigger)[0]
print("Target Fidelity (Attack Objective): %.2f%%" % np.sum((x_pred_trigger - x_target) ** 2))

Target Fidelity (Attack Objective): 64.77%


In [16]:
np.save("z_trigger_trail.npy", z_trigger)
np.save("x_target_trail.npy", x_target)
poisoned_generator.model.save("trail-mnist-dcgan")

INFO:tensorflow:Assets written to: trail-mnist-dcgan/assets


INFO:tensorflow:Assets written to: trail-mnist-dcgan/assets


# Задание

Повторить эксперимент для целевого изображения выбранного из train_image (номер обучающегося из списка Гугл-таблицы) и сгенерированного триггера из диапазона (0;59+номер обучающегося из списка Гугл-таблицы).
Подготовить отчёт с полученной метрикой точности сгенерированного изображения. Сделать выводы об изученном методе проведения атаки на GAN.

Вариант 4. 

In [25]:
# z_trigger_by_variant = np.random.uniform(low=0, high=63, size=(1,100))
# z_trigger_by_variant = np.random.randn(1, 100).astype(np.float64)
# не совсем понял сразу каким именно образом необходимо использовать триггер

z_trigger_by_variant = np.random.randn(1, 63).astype(np.float64) # использую 59 + 4 элементов
z_trigger_by_variant

array([[ 0.13296228, -0.08289853,  0.21421905,  0.69269606, -1.90818478,
         0.11653332,  1.75517417, -0.39556563, -0.93258346, -0.67356455,
         0.71453205, -1.79000392,  0.22430285, -3.20995538,  1.52358483,
        -0.06961566,  1.3655902 , -0.55869534,  0.27539348,  2.40091831,
        -1.01103512,  0.94250688, -2.22151082, -2.58576759,  0.68892136,
         0.04127457, -0.49772668, -0.00984729, -0.51766032,  0.30217683,
         0.773441  ,  0.98820885,  0.01760021,  0.81980615,  0.6877597 ,
         0.39315819, -0.51086319, -2.75203766, -0.19206908, -0.23406889,
        -1.60141394,  0.79102615,  0.01286688, -1.34851188,  0.86605917,
         0.79129498,  1.17695251, -1.16543173,  0.5382461 , -0.84286055,
        -2.87496888, -0.52850564, -0.32108899,  2.18227591,  0.32870246,
         1.44800142, -0.43728724,  0.91147449, -1.12682828,  1.11473389,
        -1.22656743,  1.35527256, -0.08885767]])

In [26]:
noise_dim = 63
capacity = 64

generator = TensorFlowV2Generator(encoding_length=noise_dim, model=make_generator_model(capacity, noise_dim))

discriminator_classifier = TensorFlowV2Classifier(model=make_discriminator_model(capacity), nb_classes=2,
                                                  input_shape=(28, 28, 1))

gan = TensorFlowV2GAN(
    generator=generator,
    discriminator=discriminator_classifier,
    generator_loss=generator_loss,
    generator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),
    discriminator_loss=discriminator_loss,
    discriminator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),
)

In [28]:
gan_attack = BackdoorAttackDGMTrailTensorFlowV2(gan=gan)

poisoned_generator = gan_attack.poison_estimator(
    z_trigger=z_trigger_by_variant, x_target=x_target[3:4], images=train_images, # выбираю 4-ое изображение
    batch_size=32, max_iter=4, lambda_g=0.1, verbose=2)



ValueError: Input 0 of layer "sequential_8" is incompatible with the layer: expected shape=(None, 63), found shape=(1, 100)

In [30]:
x_pred_trigger = poisoned_generator.model(z_trigger_by_variant)[0]
print("Target Fidelity (Attack Objective): %.2f%%" % np.sum((x_pred_trigger - x_target[3:4]) ** 2))

Target Fidelity (Attack Objective): 26.60%
